### Need >77 GB by the time CRPS cells are run

Made chunks larger so they are not 2Mb but 20Mb by unchunking `number` (ensemble member) dimension. (100-200Mb ideal) 

In [ ]:
import os
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import hwt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import properscoring as ps
import xarray as xr
from dask.distributed import Client, LocalCluster
from earth2studio.data import WB2Climatology
from earth2studio.data.mpas_ens import MPASEnsemble
from earth2studio.data.rx import SurfaceGeoPotential
from hwt import get_dynamics_model, init_times, init_times23, init_times24
from loguru import logger
from metpy.constants import g
from tqdm.notebook import tqdm
from utils import era5_varid, gefs_members, get_graphcast_output

# Set the earth2studio logger to INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

# Place where Dave cached datasets for quicker use
CACHEDIR = Path("/glade/derecho/scratch/ahijevyc/ldmason_hwt")

graphcast_subdir = "graphcast"

PLOTDIR = Path("plots") / graphcast_subdir
if not os.path.exists(PLOTDIR):
    os.makedirs(PLOTDIR, exist_ok=True)

# sorted list of unique years in init_times
years = sorted(list(set(i.year for i in init_times)))

forecast_length = 240
# Truncate members
member_cutoff = 10  # 1, 5, 10, or None

In [ ]:
parallel = True
if parallel:
    try:
        client.close()
    except NameError:
        pass

    try:
        cluster.close()
    except NameError:
        pass
    # Create a local cluster with workers (CPUs)
    cluster = LocalCluster()
    client = Client(cluster)

    print(client.dashboard_link)

In [ ]:
# These variables may be changed between temperature and height
shortName = "z"
isobaricInhPa = 500
units = "m"


def local_path_from_url(gefsdir, file_url_str):
    url_path = Path(file_url_str)
    file_path = gefsdir.joinpath(*url_path.parts[-5:])
    os.makedirs(file_path.parent, exist_ok=True)
    return file_path


def open_grib_dataset(path):
    ds = xr.open_dataset(
        path,
        engine="cfgrib",
        backend_kwargs={"errors": "ignore"},
        filter_by_keys={
            "typeOfLevel": "isobaricInhPa",
            "level": isobaricInhPa,
            "shortName": "gh" if shortName == "z" else shortName,
        },
        decode_timedelta=True,
        chunks={},  # chunking can help reduce memory usage
    )
    if shortName == "z":
        ds = ds.rename(gh="z")
    return ds


def ai_ifiles(pangu_dir, model, init_time, mem, forecast_length):
    # return list of files at different lead times.
    if init_time > pd.to_datetime("20250101"):
        file_path = (
            pangu_dir / init_time.strftime("%Y%m%d%H") / f"ens{mem}" / f"{model}_forecast_data"
        )
        ifiles = [
            file_path / f"{model}_ens{mem}_pred_{i:03d}.nc"
            for i in range(24, forecast_length + 1, 24)
        ]
    else:
        control_or_perturbation = "p" if mem > 0 else "c"
        file_path = (
            pangu_dir / init_time.strftime("%Y%m%d%H") / f"{control_or_perturbation}{mem:02d}"
        )
        ifiles = [
            file_path / f"{model}_gefs_pred_{i:03d}.nc" for i in range(24, forecast_length + 1, 24)
        ]

    return ifiles

In [ ]:
def get_nested_files(ai_dir, init_times, forecast_length, nmem=51):
    # derive model from ai_dir
    model = ai_dir.name.rstrip("_realtime")
    # Create a nested list where list[init_time][member] = [files_for_all_forecast_hours]
    nested_files = []
    for init_time in init_times:
        # List for all members for this init_time
        time_specific_files = []
        for mem in range(nmem):
            # List of all forecast hour files for this specific member
            lead_times = ai_ifiles(ai_dir, model, init_time, mem, forecast_length)
            if all([os.path.exists(f) for f in lead_times]):
                time_specific_files.append(lead_times)
            else:
                print(f"not all lead times present for {model} {mem} {init_time}")

        if len(time_specific_files) != nmem:
            print(f"only {len(time_specific_files)}/{nmem} {model} {init_time} files")
            continue
        nested_files.append(time_specific_files)

    return nested_files


def merge_nested_files(nested_files, shortName, isobaricInhPa, units):
    # The channel label we want to select
    channel_label = f"{shortName}{isobaricInhPa}"

    nmem = len(nested_files[0])
    ds = (
        xr.open_mfdataset(
            nested_files,
            combine="nested",
            concat_dim=["init_time", "number", "prediction_timedelta"],
            decode_timedelta=True,
            chunks="auto",  # Use 'auto' for better performance with dask
        )
        # Rename dimensions and coordinates at the start
        .rename(
            {
                "init_time": "initialization_time",
                "prediction_timedelta": "step",
                "lat": "latitude",
                "lon": "longitude",
                "__xarray_dataarray_variable__": shortName,  # Rename the main variable
            }
        )
        # Assign the integer coordinate for the 'number' dimension
        .assign_coords(number=range(nmem))
        # Convert 'step' dimension from timedelta to integer forecast hours
        .pipe(
            lambda ds: ds.assign_coords(
                forecast_hour=("step", ds["step"].data / np.timedelta64(1, "h"))
            ).swap_dims({"step": "forecast_hour"})
        )
        # Calculate the valid_time coordinate
        .assign_coords(valid_time=lambda ds: ds.initialization_time + ds.step)
        # Select the desired channel by its label (more readable)
        .sel(channel=channel_label)
        # Add the pressure level as a non-dimension coordinate
        .assign_coords(isobaricInhPa=isobaricInhPa)
    )

    if shortName == "z":
        if "units" in ds["z"].attrs:
            # Let metpy take care of the units
            ds = ds.metpy.quantify()
            print("divide by g (as Quantity with units)")
            ds["z"] /= g
            ds = ds.metpy.dequantify()
        else:
            # no units, like in fengwu
            print("divide by g (no units)")
            ds["z"] /= g.m
            ds["z"].attrs.update({"units": units})
    if "units" in ds[shortName].attrs:
        assert (
            ds[shortName].attrs["units"] == units
        ), f"expected units {units}. got {ds[shortName].attrs['units']}"
    return ds

In [ ]:
if 2025 in years:
    # Output from real-time runs in Ryan's directory
    pangu_dir = Path("/glade/derecho/scratch/sobash/pangu_realtime")
    nested_files = get_nested_files(pangu_dir, init_times, forecast_length)
    ds_pangu = merge_nested_files(nested_files, shortName, isobaricInhPa)

    fengwu_dir = Path("/glade/derecho/scratch/sobash/fengwu_realtime")
    nested_files = get_nested_files(fengwu_dir, init_times, forecast_length)
    ds_fengwu = merge_nested_files(nested_files, shortName, isobaricInhPa)
else:
    ofile = CACHEDIR / f"pangu.{isobaricInhPa}{shortName}.{years}.{forecast_length:03d}h.zarr"
    if os.path.exists(ofile):
        print(f"open existing {ofile}")
        ds_pangu = xr.open_zarr(ofile, decode_timedelta=False)
    else:
        print(f"making {ofile}")
        # Output from older dates (HWT2023/4)
        # Nested list
        #                         init0          , ... ,           initn
        # model_runs = [[gec0, gep1, ... , gep30], ... , [gec0, gep1, ... , gep30]]
        model_runs = []
        for init_time in init_times:
            idir = Path(
                "/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather"
            ) / init_time.strftime("%Y%m%d%H")
            ifiles = sorted(list(idir.glob("g??[0-9][0-9].grib")))
            if len(ifiles) == 31:
                model_runs.append(ifiles)
            else:
                logging.warning(f"{init_time} has {len(ifiles)}/31 pangu files in {idir}")
        ds = xr.open_mfdataset(
            model_runs,
            engine="cfgrib",
            backend_kwargs={"errors": "ignore"},
            filter_by_keys={
                "typeOfLevel": "isobaricInhPa",
                "level": isobaricInhPa,
                "shortName": shortName,  # Don't worry about z being called gh. It's called z.
            },
            decode_timedelta=True,
            chunks={},  # chunking can help reduce memory usage
            combine="nested",
            concat_dim=["time", "number"],
        )
        ds = ds.rename(time="initialization_time")
        # Convert 'step' dimension from timedelta to integer forecast hours
        ds = ds.assign_coords(
            forecast_hour=("step", ds["step"].data / np.timedelta64(1, "h"))
        ).swap_dims({"step": "forecast_hour"})
        # Just multiples of 24 hours, please.
        ds = ds.sel(forecast_hour=range(0, forecast_length + 1, 24))
        if shortName == "z":
            # Let metpy take care of units (must be Quantity first)
            # m^2/s^2 -> m
            ds = ds.metpy.quantify()
            ds["z"] /= g
            ds = ds.metpy.dequantify()
        ds_pangu = ds
        ds_pangu.to_zarr(ofile)
    ofile = CACHEDIR / f"fengwu.{isobaricInhPa}{shortName}.{years}.{forecast_length:03d}h.zarr"
    if os.path.exists(ofile):
        print(f"open existing {ofile}")
        # fengwu runs out of mem if chunks['number'] = 1. Set to -1.
        ds_fengwu = xr.open_zarr(ofile, decode_timedelta=False, chunks={"number": -1})
    else:
        print(f"making {ofile}")
        fengwu_dir = Path("/glade/derecho/scratch/ahijevyc/ai-models/output/fengwu")
        nested_files = get_nested_files(fengwu_dir, init_times, forecast_length, nmem=31)
        ds_fengwu = merge_nested_files(nested_files, shortName, isobaricInhPa, units)
        ds_fengwu.to_zarr(ofile)

    # weathernext zarr has multiple variables/isobaricInhPa, as opposed to pangu and fengwu zarr files.
    ofile = CACHEDIR / f"weathernext.{years}.{forecast_length:03d}h.zarr"
    idir = Path("/glade/derecho/scratch/sobash/weathernext")
    ifiles = [
        [
            idir / f"{d:%Y%m%d%H}/weathernext_{d:%Y%m%d%H}_{fhr:03d}.nc"
            for fhr in range(24, forecast_length + 1, 24)
        ]
        for d in init_times
    ]

    def openwxnext(ifiles, var, isobaricInhPa):
        ds = (
            xr.open_mfdataset(
                ifiles,
                combine="nested",
                concat_dim=["time", "prediction_timedelta"],
                chunks={"number": -1},
            )[var]
            .sel(level=[isobaricInhPa])  # pass list to preserve isobaricInhPa coord for zarr
            .rename(
                level="isobaricInhPa",
                sample="number",
                lat="latitude",
                lon="longitude",
                time="initialization_time",
            )
        )
        ds["forecast_hour"] = ds["prediction_timedelta"] / pd.to_timedelta(1, unit="h")
        ds["valid_time"] = ds["initialization_time"] + ds["prediction_timedelta"]
        ds = ds.swap_dims(prediction_timedelta="forecast_hour")
        return ds

    if not os.path.exists(ofile):
        print(f"making {ofile}")
        ds_z = openwxnext(ifiles, "geopotential", 500).rename("z")
        ds_t = openwxnext(ifiles, "temperature", 850).rename("t")
        # Tried concat, but won't handle two variables.
        # Tried 'compat="override"' or 'join="exact"' since coordinates
        # (except isobaricInhPa) are identical to save compute time.
        # But AlignmentError: cannot align join='exact' where index not equal: 'isobaricInhPa' ('isobaricInhPa',)
        ds_wxnext = xr.combine_by_coords([ds_z, ds_t], compat="override").sortby(
            "latitude", ascending=False
        )
        ds_wxnext.to_zarr(ofile)
    # Thought by saving with chunks this wouldn't be necessary, but it is.
    # Unchunked forecast_hour too since chunks were so tiny (2.12MiB). Now they are 21.2MiB)
    print(f"open existing {ofile}")
    ds_wxnext = xr.open_zarr(ofile, chunks={"number": -1, "forecast_hour": -1})
    ds_wxnext = ds_wxnext[[shortName]].sel(isobaricInhPa=isobaricInhPa)
    # convert geopotential (m2/s2) to geopotential height (m)
    if shortName == "z":
        ds_wxnext["z"] = ds_wxnext["z"] / g.m

ds_wxnext

In [ ]:
ofile = CACHEDIR / f"{graphcast_subdir}.{years}.zarr"
if not ofile.exists():
    ds_graphcast = get_graphcast_output(shortName, isobaricInhPa, subdir=graphcast_subdir)
    print(f"making {ofile}")
    ds_graphcast.to_zarr(ofile)
ds_graphcast = xr.open_zarr(ofile, chunks={"number": -1})
ds_graphcast = ds_graphcast.sel(forecast_hour=slice(None, forecast_length))
ds_graphcast

In [ ]:
ofile = CACHEDIR / "mpas15-3km.zarr"
# native horizontal mpas mesh
if not ofile.exists():
    datadir = Path("/glade/campaign/mmm/parc/schwartz")
    # 2023
    mpas_datasrc = MPASEnsemble(
        grid_path=datadir / "MPAS/15-3km_mesh/grid_mesh/x5.6488066.grid_CONUS.nc",
        data_path=f"{datadir}/HWT2023/mpas",
    )
    for initialization_time in tqdm(init_times23):
        objs = []
        for forecast_hour in range(0, int(5.5 * 24) + 1, 24):
            ds = mpas_datasrc(initialization_time, variables=["t850", "z500"], fhr=forecast_hour)
            objs.append(
                ds.expand_dims(
                    initialization_time=[initialization_time], forecast_hour=[forecast_hour]
                ).squeeze("Time")
            )
        ds = xr.combine_by_coords(objs, combine_attrs="override")
        if os.path.exists(ofile):
            # consolidated=False because it's not official part of version 3 and local storage means it won't help much
            ds.to_zarr(ofile, mode="a", append_dim="initialization_time", consolidated=False)
        else:
            ds.to_zarr(ofile, mode="w", consolidated=False)
    # 2024
    mpas_datasrc = MPASEnsemble(
        grid_path=datadir / "MPAS/15-3km_mesh/grid_mesh/x5.6488066.grid_CONUS.nc",
        data_path=f"{datadir}/HWT2024/mpas",
    )
    for initialization_time in tqdm(init_times24):
        objs = []
        for forecast_hour in range(0, int(5.5 * 24) + 1, 24):
            ds = mpas_datasrc(initialization_time, variables=["t850", "z500"], fhr=forecast_hour)
            objs.append(
                ds.expand_dims(
                    initialization_time=[initialization_time], forecast_hour=[forecast_hour]
                ).squeeze("Time")
            )
        ds = xr.combine_by_coords(objs, combine_attrs="override")
        ds.to_zarr(ofile, mode="a", append_dim="initialization_time", consolidated=False)

ds_mpas = (
    xr.open_zarr(ofile, consolidated=False)["__xarray_dataarray_variable__"]
    .to_dataset(dim="variable")
    .rename(
        lat="latitude",
        lon="longitude",
        member="number",
        t850="t",
        z500="z",
    )
    .assign_coords(
        valid_time=lambda x: x["initialization_time"] + x["forecast_hour"] * pd.Timedelta("1h")
    )
    .isel(latitude=slice(None, None, -1))
)
ds_mpas["z"] = ds_mpas["z"] / g.m
ds_mpas

In [ ]:
zarrfile = CACHEDIR / "fv3.zarr"
if not zarrfile.exists():
    ds_fv3: xr.Dataset = get_dynamics_model(
        "fv3",
        forecast_hours=range(24, 204 + 1, 24),
        ens_size=10,
        vars_dict={"TMP850": "t", "HGT500": "z"},
        init_times=init_times,
        output_grid=ds_pangu,
        CACHEDIR=CACHEDIR,
    )
ds_fv3 = xr.open_zarr(
    zarrfile, chunks={"number": -1, "forecast_hour": -1, "initialization_time": 10}
)
ds_fv3

In [ ]:
if False:
    # interpolated to lat/lon "interp_mpas_3km_*.nc" files
    zarrfile = CACHEDIR / "mpas_interp.zarr"
    if not zarrfile.exists():
        # get_dynamics_model also saves f"{model}.zarr"
        # Reads interpolated files f"interp_{model}_3km_{init_time:%Y%m%d%H}_mem{mem}_f{fhr:03d}.nc"
        # Not native hybrid levels.
        ds_mpas_interp: xr.Dataset = get_dynamics_model(
            model="mpas",
            forecast_hours=range(0, 132 + 1, 24),  # MPAS only goes 5.5d
            ens_size=5,  # only 5/10 members interpolated to lat/lon/pressure_level and saved
            vars_dict={"temperature_850hPa": "t", "height_500hPa": "z"},
            init_times=init_times,
            output_grid=ds_pangu,
            CACHEDIR=CACHEDIR,
        )
    ds_mpas_interp = xr.open_zarr(zarrfile)
    ds_mpas_interp
    
    sel = dict(
        initialization_time="20240501",
        forecast_hour=120,
        number=2,
        longitude=slice(230, 290),
        latitude=slice(52, 21),
    )
    fig, axes = plt.subplots(ncols=3, figsize=(18, 4))
    vmin = 210
    ds_mpas.sel(sel).t.plot(ax=axes[0], vmin=vmin)
    ds_mpas_interp.sel(sel)[shortName].plot(ax=axes[1], vmin=vmin)
    qm = (ds_mpas - ds_mpas_interp).sel(sel)[shortName].plot(ax=axes[2], vmin=-1, vmax=1)
    qm.figure.suptitle("native mpas mesh vs lat-lon interpolated")

In [ ]:
gefsdir = Path("/glade/derecho/scratch/ahijevyc/tmp/gefs")

ofile = CACHEDIR / f"gefs.{isobaricInhPa}{shortName}.{years}.{forecast_length:03d}h.zarr"

# Try saved ds_gefs. I tried saving zarr with compute=False but when I loaded, values were zero.
if os.path.exists(ofile):
    print(f"open existing {ofile}")
    ds_gefs = xr.open_zarr(ofile, decode_timedelta=False, chunks={"number": -1})
else:
    existing_files = set(p.resolve() for p in gefsdir.glob("*/??/atmos/pgrb2ap5/*pgrb2a.0p50.f???"))

    # Base URL for the public NOAA GEFS S3 bucket
    base_url = "https://noaa-gefs-pds.s3.amazonaws.com"

    # ===================================================================
    # PHASE 1: DEFINE FILE REQUIREMENTS
    # ===================================================================
    print("--- Phase 1: Defining all required file paths ---")
    required_files = []

    for init_time in init_times:
        for member in gefs_members:
            for fhr in range(0, forecast_length + 1, 24):
                fhr_str = f"{fhr:03d}"
                s3_filename = f"{member}.t{init_time:%H}z.pgrb2a.0p50.f{fhr_str}"
                file_path_on_s3 = (
                    f"gefs.{init_time:%Y%m%d}/{init_time:%H}/atmos/pgrb2ap5/{s3_filename}"
                )
                url = f"{base_url}/{file_path_on_s3}"
                local_path = local_path_from_url(gefsdir, url)
                required_files.append({"url": url, "local_path": local_path})

    print(f"✅ Defined {len(required_files)} files required for analysis.\n")
    print(f"✅ Data saved in: {gefsdir}")
    print("-" * 50)
    datasets = []

    print(f"making {ofile}")
    for required_file in tqdm(required_files):
        url = required_file["url"]
        local_file_path = required_file["local_path"]
        if local_file_path in existing_files:
            # print(f"   -> 🟢 File already exists. Skipping.")
            pass
        else:
            print(local_file_path)
            # Ensure the destination directory exists before downloading
            Path(local_file_path).parent.mkdir(parents=True, exist_ok=True)
            print(f"   -> ⬇️  Attempting to download: {url}")
            download_file(url, local_file_path)
        ds_gefs = open_grib_dataset(local_file_path)
        datasets.append(ds_gefs)

    ds_gefs = xr.combine_nested(datasets, concat_dim=["time"])

    itime_coords = ds_gefs["valid_time"] - ds_gefs["step"]
    ds_gefs = ds_gefs.assign_coords(initialization_time=itime_coords)
    ds_gefs = ds_gefs.groupby(["initialization_time", "step", "number"]).first()

    step_as_hours = ds_gefs["step"].data / pd.to_timedelta("1h")
    ds_gefs = ds_gefs.assign_coords(forecast_hour=("step", step_as_hours))
    ds_gefs = ds_gefs.swap_dims(step="forecast_hour")
    ds_gefs = ds_gefs.assign_coords(valid_time=ds_gefs.initialization_time + ds_gefs.step)
    ds_gefs.to_zarr(ofile)

ds_gefs

In [ ]:
era5_dir = Path("/glade/campaign/collections/rda/data/d633000/e5.oper.an.pl")

# tack on extra truth times to compare to the forecast valid times
truth_times = init_times
for init_time in init_times:
    valid_times = pd.date_range(init_time, init_time + pd.to_timedelta(f"{forecast_length}h"))
    truth_times = truth_times.union(valid_times)

era5_files = [
    era5_dir
    / i.strftime("%Y%m")
    / f"e5.oper.an.pl.128_{era5_varid[shortName]}_{shortName}.ll025sc.{i:%Y%m%d00}_{i:%Y%m%d23}.nc"
    for i in truth_times
]
truth = (
    xr.open_mfdataset(era5_files)  # tried chunks time=10 but shuffle-sorter took forever
    .sel(level=isobaricInhPa)
    .rename(time="valid_time", level="isobaricInhPa")
    .rename({shortName.upper(): shortName})  # T->t, Z->z. uppercase shortName to lower
    .drop_vars("utc_date")
)
# era5 available every hour. just return 0z please
truth = truth.sel(valid_time=(truth.valid_time.dt.hour == 0))

# convert geopotential to geopotential height
if shortName == "z":
    truth["z"] = truth["z"] / g.m
truth

In [ ]:
ds_mpas

In [ ]:
# Apply geographic_sel to ds_gefs and mask to HWT region.
geographic_sel = dict(latitude=slice(60, 20), longitude=slice(220, 300))
# Use GEFS coords to select from all other Datasets.

print("gefs")
ds_gefs = ds_gefs.sel(geographic_sel).where(hwt.mask0p5).load()
gefs0p5 = dict(latitude=ds_gefs.latitude, longitude=ds_gefs.longitude)

# Select ds_gefs coords from other datasets.
print("mpas")
ds_mpas = ds_mpas.sel(gefs0p5).where(hwt.mask0p5).load()
print("fv3")
ds_fv3 = ds_fv3.sel(gefs0p5).where(hwt.mask0p5).load()
print("pangu")
ds_pangu = ds_pangu.sel(gefs0p5).where(hwt.mask0p5).load()
print("fengwu")
ds_fengwu = ds_fengwu.sel(gefs0p5).where(hwt.mask0p5).load()
print("graphcast")
ds_graphcast = ds_graphcast.sel(gefs0p5).where(hwt.mask0p5).load()
print("wxnext")  # subset of "HWT" domain, 60-20N 220-300E.
# Reindex to larger domain so you can use same mask. method=None to pad with NaNs
ds_wxnext = ds_wxnext.reindex(gefs0p5).where(hwt.mask0p5)#.load()
print("truth")
truth = truth.sel(gefs0p5).where(hwt.mask0p5).load()

In [ ]:
# This code assumes the following variables are already loaded in your script:
# ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengw, ds_graphcast, : xr.Dataset objects with model data
# truth: xr.Dataset with the ground truth data
# shortName: A string representing the variable name (e.g., 't' for temperature)
# units: A string for the y-axis label (e.g., 'K' for Kelvin)
# isobaricInhPa: An integer representing the pressure level (e.g., 500)


# These will store the handles and labels for the shared legend
handles = []
labels = []

# --- Define models and years to iterate over ---
models_data = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
model_titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

# --- Plotting Setup ---
# Create a nxm grid of subplots: n rows (for years), m columns (for models)
fig, axes = plt.subplots(nrows=len(years), ncols=len(models_data), figsize=(20, 7.5), sharey='row')


# --- Main Loop: Iterate over years and models ---
# Loop over each year to create a row of plots
for row_idx, year in enumerate(years):
    # Loop over each model to create a plot in the corresponding column
    for col_idx, (ds_model, title) in enumerate(zip(models_data, model_titles)):
        ax = axes[row_idx, col_idx]

        # --- Data processing steps (as before) ---
        ensemble_mean = ds_model.mean(dim=["number", "latitude", "longitude"])
        nmembers = ds_model.sizes["number"]
        stacked_ds = ensemble_mean.stack(point=("initialization_time", "forecast_hour"))
        tidy_ds = stacked_ds.set_index(point=["initialization_time", "valid_time"])
        plot_ready_full = tidy_ds[shortName].unstack("point")

        # --- Filter data for the specific year ---
        # Use xr's .sel() method for selection instead of pandas' .index
        plot_ready = plot_ready_full.sel(valid_time=plot_ready_full.valid_time.dt.year == year)
        truth_filtered = truth.sel(valid_time=truth.valid_time.dt.year == year)

        # --- Plotting ---
        # Plot the model forecast lines
        # Use .size to check if an xr DataArray has data
        if not plot_ready.size:
            print("skip", year, title, "no data")
            continue
        plot_ready.plot.line(
            x="valid_time",
            hue="initialization_time",
            ax=ax,
            add_legend=False,
        )
        # Plot the truth data
        assert truth_filtered[shortName].size > 0
        truth_filtered[shortName].mean(dim=["latitude", "longitude"]).plot.line(
            ax=ax, x="valid_time", marker="o", color="k", markersize=3
        )

        # --- Axis formatting ---
        # Set the model title for the top row plots only
        if row_idx == 0:
            ax.set_title(f"{title} ({nmembers} mem)", fontsize=14)

        # Set the y-axis label to include the year on the leftmost plot of each row
        if col_idx == 0:
            ax.set_ylabel(f"{year}\n{shortName} [{units}]", fontsize=12)
        else:
            ax.set_ylabel("")  # Hide y-label for other plots in the row

        ax.set_xlabel("Valid Time")
        ax.grid(True, linestyle="--", alpha=0.6)

        # --- Capture handles and labels ONCE from the first valid plot ---
        if not handles and ax.get_lines():
            all_lines = ax.get_lines()
            truth_line = all_lines[-1]  # The last line plotted is the 'truth' line
            forecast_lines = all_lines[:-1]  # The other lines are the forecasts

            handles.extend(forecast_lines)
            handles.append(truth_line)

            # Create labels for forecast lines
            forecast_labels = [
                pd.to_datetime(t).strftime("%Y-%m-%d")
                for t in plot_ready.initialization_time.values
            ]
            labels.extend(forecast_labels)
            labels.append("Truth")  # Add the label for the truth line

# --- Final Figure Adjustments ---
# Add a shared legend to the right of the figure
fig.legend(
    handles,
    labels,
    title="Initialization",
    loc="center left",
    bbox_to_anchor=(1.0, 0.5),
    ncols=len(years),
)

# Add a main title for the entire figure
fig.suptitle(f"{isobaricInhPa}-hPa {shortName} Forecasts by Year", fontsize=16)

# Adjust layout to make room for the legend and suptitle
fig.tight_layout()

plt.savefig(PLOTDIR / f"ensmean_{isobaricInhPa}{shortName}_{years}.png", dpi=200)

In [ ]:
# --- Configuration ---
models_data = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
model_titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

fig, axes = plt.subplots(
    nrows=len(years), ncols=len(models_data), figsize=(20, 7.5), sharex=False, sharey="row"
)

fig_handles = []
fig_labels = []
# Create color cycle to pull from--one color for each init_time
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
# --- Main Loop ---
for row_idx, year in enumerate(years):
    for col_idx, (ds_model, title) in enumerate(zip(models_data, model_titles)):
        ax = axes[row_idx, col_idx]

        # 1. Filter and reduce spatially - keep it as a Dask graph for now
        ds_year = ds_model.sel(initialization_time=ds_model.initialization_time.dt.year == year)

        # 2. COMPUTE: This is the heavy lift.
        # Returns a small in-memory xarray DataArray of (init_time, number, forecast_hour)
        plot_data = ds_year[shortName].mean(dim=["latitude", "longitude"]).compute()

        # 3. Plot each initialization group
        for i, init_time in enumerate(plot_data.initialization_time):
            group = plot_data.sel(initialization_time=init_time)

            group_color = colors[i % len(colors)]

            # Using xarray's vectorized line plot for all members in this init_time
            # Removed add_labels=False to avoid the AttributeError
            lines = group.plot.line(
                ax=ax,
                x="valid_time",
                hue="number",
                add_legend=False,
                alpha=0.5,
                linewidth=0.8,
                color=group_color,
            )

            # 4. Legend Capture (Only from first subplot)
            # 'lines' is a list of Line2D objects. We just need one to represent the group.
            if col_idx == 0:
                label_str = pd.to_datetime(init_time.values).strftime("%Y-%m-%d")
                if label_str not in fig_labels:
                    # Set a specific color for the whole group to keep legend clean
                    group_color = lines[0].get_color()
                    fig_handles.append(lines[0])
                    fig_labels.append(label_str)

        # 5. Plot Truth
        truth_year = truth.sel(valid_time=truth.valid_time.dt.year == year)
        truth_mean = truth_year[shortName].mean(dim=["latitude", "longitude"]).compute()

        # Plotting truth and capturing the handle for the legend
        t_line = ax.plot(
            truth_mean.valid_time,
            truth_mean.values,
            color="black",
            marker="o",
            markersize=3,
            linewidth=2,
            label="Truth",
        )

        if col_idx == 0 and "Truth" not in fig_labels:
            fig_handles.append(t_line[0])
            fig_labels.append("Truth")

        # --- Formatting ---
        if row_idx == 0:
            ax.set_title(f"{title} ({ds_model.sizes['number']} mem)")

        # Clean up x-axis labels to avoid overlaps
        ax.set_xlabel("")
        if row_idx == len(years) - 1:
            ax.set_title("")
            ax.set_xlabel("Valid Time")

        if col_idx == 0:
            ax.set_ylabel(f"{year}\n{shortName} [{units}]")
        else:
            ax.set_ylabel("")

        ax.grid(True, linestyle="--", alpha=0.5)

# --- Final Adjustments ---
fig.legend(
    fig_handles,
    fig_labels,
    title="Initialization",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=2,
)

fig.suptitle(f"{isobaricInhPa}-hPa {shortName} Forecasts by Year", fontsize=16)
plt.tight_layout()  # Make room for the legend

plt.savefig(PLOTDIR / f"spaghetti_{isobaricInhPa}{shortName}_{years}.png", dpi=200)

In [ ]:
ds_fengwu[shortName].mean(dim=["latitude", "longitude", "forecast_hour"]).plot()

In [ ]:
fig, axes = plt.subplots(ncols=7, nrows=2, figsize=(18, 6), sharex=True, sharey="row")
max_y_top = 0
for col_idx, ((ax, ax_bot), ds_model, title) in enumerate(
    zip(
        axes.T,
        [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext],
        ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"],
    )
):
    ds_model = ds_model.assign_coords(day=ds_model.forecast_hour / 24)

    # select 0.5-deg grid of GEFS or pangu fhr=0 has higher RMSE
    # (because I didn't linearly interpolate pl fields from 0.5 to 0.25-deg grid in pangu input?)
    ds_model = ds_model.sel(
        latitude=ds_gefs.latitude, longitude=ds_gefs.longitude, method="nearest"
    )
    nmembers = ds_model.sizes["number"]
    # average over initialization_time, spatial dims. std over ensemble (number).
    ensemble_spread = (
        ds_model[shortName]
        .std(dim="number", ddof=1)
        .mean(dim=["initialization_time", "latitude", "longitude"])
    )
    # error = ensemble mean - truth
    # Line up truth and forecast along `valid_time`.
    # if you get error about truth not having all valid_times,
    # you probably forgot to filter based on forecast_length
    error = ds_model.mean(dim="number") - truth.sel(valid_time=ds_model["valid_time"])
    se = error[shortName] ** 2
    mse = se.mean(dim=["latitude", "longitude", "initialization_time"])
    rmse = np.sqrt(mse)
    rmse.plot.line(ax=ax, x="day", marker="o", label="rmse")
    ensemble_spread.plot.line(ax=ax, x="day", marker="o", label="spread")
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.set_title(f"{title} ({nmembers}) {isobaricInhPa}-hPa {shortName}", fontsize=14)

    current_max = max(rmse.max(), ensemble_spread.max())
    max_y_top = max(max_y_top, current_max)

    ratio = ensemble_spread / rmse
    ratio.plot.line(ax=ax_bot, x="day", color="black", marker="s", label="ratio")
    # Reference line for perfect spread
    ax_bot.axhline(1, color="red", linestyle="-", alpha=0.3)
    ax_bot.grid(True, linestyle="--", alpha=0.6)
    ax_bot.set_ylim(0, 1.5)  # Ratios usually hover around 1.0

for ax in axes[0, :]:
    ax.set_ylim(0, max_y_top * 1.05)  # Added 5% padding for headspace
plt.tight_layout()
ofile = PLOTDIR / f"rmse_spread_{isobaricInhPa}{shortName}_{years}.png"
plt.savefig(ofile, dpi=200)
print(ofile)

In [ ]:
import seaborn as sns

# Set the global aesthetic
sns.set_theme(style="ticks")
fig, axes = plt.subplots(ncols=7, figsize=(18, 3), sharex=True, sharey=True)
forecast_hour_to_plot = 132

models = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

for ax, ds_model, title in zip(axes, models, titles):
    # --- 1. Processing & Calculation ---
    ds_sub = ds_model.sel(
        latitude=ds_gefs.latitude,
        longitude=ds_gefs.longitude,
        method="nearest",
        forecast_hour=forecast_hour_to_plot,
    )

    spread = ds_sub[shortName].std(dim="number", ddof=1).mean(dim=["latitude", "longitude"])
    ens_mean = ds_sub[shortName].mean(dim="number")
    obs = truth[shortName].sel(valid_time=ds_sub["valid_time"])
    rmse = np.sqrt(((ens_mean - obs) ** 2).mean(dim=["latitude", "longitude"]))

    # --- 2. Tidying & Rolling Mean ---
    def tidy_with_rolling(da, label, window=7):
        df = da.to_dataframe(name="value").reset_index()
        df["year"] = df["initialization_time"].dt.year.astype(str)
        df["day"] = df["initialization_time"].dt.dayofyear
        df["metric"] = label

        df = df.sort_values("initialization_time")

        # Calculate rolling mean per year to avoid bleeding 2023 into 2024
        df["rolling_val"] = df.groupby("year")["value"].transform(
            lambda x: x.rolling(window=window, center=True).mean()
        )
        return df

    df_plot = pd.concat(
        [tidy_with_rolling(rmse, "RMSE"), tidy_with_rolling(spread, "Spread")]
    ).reset_index(drop=True)

    # --- 3. Plotting ---
    # Plot raw data as faint lines (alpha=0.3)
    sns.lineplot(
        data=df_plot,
        x="day",
        y="value",
        hue="year",
        style="metric",
        hue_order=["2023", "2024"],
        alpha=0.25,
        lw=1,
        legend=False,
        ax=ax,
    )

    # Plot rolling mean as thick, solid lines
    sns.lineplot(
        data=df_plot,
        x="day",
        y="rolling_val",
        hue="year",
        style="metric",
        hue_order=["2023", "2024"],
        lw=2.5,
        ax=ax,
    )

    # --- 4. Aesthetics ---
    ax.set_title(f"{title} {forecast_hour_to_plot}h {isobaricInhPa}-hPa {shortName}")
    ax.grid(True, axis="y", linestyle=":", alpha=0.6)
    ax.set_xlabel("Day of Year" if ax in axes[2:4] else "")
    ax.set_ylabel(f"{shortName} ({units})" if ax == axes[0] else "")

    if ax != axes[-1]:
        ax.get_legend().remove()
    else:
        # Customizing legend labels for clarity
        handles, labels = ax.get_legend_handles_labels()
        # Filter legend to only show the main categories
        ax.legend(handles, labels, loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

sns.despine()
plt.tight_layout()
plt.savefig(PLOTDIR / f"rmse_spread_timeseries_{isobaricInhPa}{shortName}_{years}.png", dpi=200)

In [ ]:
fig, axes = plt.subplots(ncols=7, figsize=(18, 3), sharex=True, sharey=True)

for ax, ds_model, title in zip(
    axes,
    [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext],
    ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"],
):
    # Assign 'day' coordinate for plotting
    ds_model = ds_model.assign_coords(day=ds_model.forecast_hour / 24)
    print(title)

    # Align the truth dataset to the forecast's valid_time for accurate comparison
    # This ensures that for each forecast point, we have a corresponding observation.
    aligned_truth = truth[shortName].sel(valid_time=ds_model["valid_time"])

    # --- RMSE and Spread ---
    # Calculate ensemble spread: std dev across members, then averaged
    ensemble_spread = (
        ds_model[shortName]
        .std(dim="number", ddof=1)
        .mean(dim=["initialization_time", "latitude", "longitude"])
    )

    # Calculate RMSE: error of ensemble mean vs. truth
    error = ds_model.mean(dim="number") - aligned_truth
    se = error[shortName] ** 2
    mse = se.mean(dim=["latitude", "longitude"])
    rmse = np.sqrt(mse).mean(dim="initialization_time")

    # Calculate CRPS. The 'number' dimension is now correctly the last axis.
    # via apply_ufunc (Avoids .values)
    # This keeps the calculation 'lazy' and memory-efficient
    crps = xr.apply_ufunc(
        ps.crps_ensemble,  # The function to apply
        aligned_truth,  # Argument 1 (observations)
        ds_model[shortName],  # Argument 2 (forecasts)
        input_core_dims=[[], ["number"]],  # 'number' is the ensemble dim in forecast
        join="inner",  # handle coord mismatches
        dask="parallelized",  # Allows Dask to handle it in chunks
        output_dtypes=[float],
        dask_gufunc_kwargs={"allow_rechunk": True},
    )

    # 5. Aggregate CRPS and Trigger Computation
    # We call .compute() or .load() only on the small, reduced result
    crps_mean = crps.mean(dim=["initialization_time", "latitude", "longitude"]).compute()

    # --- Plotting ---
    # Plot the original and new metrics
    logging.info("plot")
    rmse.plot.line(ax=ax, x="day", marker="o", label="RMSE")
    ensemble_spread.plot.line(ax=ax, x="day", marker="o", label="Spread")
    crps_mean.plot.line(ax=ax, x="day", marker="o", label="CRPS", linestyle="--")

    # --- Formatting ---
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.set_title(f"{title} Forecast\n{isobaricInhPa}-hPa {shortName}", fontsize=12)
    ax.legend()
    ax.set_ylabel(f"Score [{units}]")
    ax.set_xlabel("Forecast Lead Time (days)")

# Adjust layout
plt.tight_layout()

In [ ]:
models_data = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
model_titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]
# We set sharex=False and sharey=False because the scales differ wildly between
# a 10-member and 64-member ensemble.
fig, axes = plt.subplots(2, 4, figsize=(12, 6), sharex=False, sharey=False)
axes = axes.flatten()

for i, (ds_model, title) in enumerate(zip(models_data, model_titles)):
    ax = axes[i]
    print(f"Processing {title} (Members: {ds_model.number.size})...")

    # 1. Dynamic Member Count
    n_members = ds_model.number.size
    # The ideal probability for a perfectly calibrated ensemble
    ideal_freq = 1 / (n_members + 1)

    # ds_gefs has 0.5deg grid; select only those points from ds_model and truth.
    ds_model = ds_model.sel(
        latitude=ds_gefs.latitude, longitude=ds_gefs.longitude, method="nearest"
    )

    aligned_truth = truth[shortName].sel(
        valid_time=ds_model["valid_time"],
        latitude=ds_gefs.latitude,
        longitude=ds_gefs.longitude,
        method="nearest",
    )

    # 1. Ensure we are working with raw numpy arrays to avoid xarray/dask overhead
    # We flatten the spatial/temporal dims but keep the 'number' (ensemble) dim
    obs = aligned_truth.values.flatten()  # shape: (N,)
    fct = ds_model[shortName].values.reshape(-1, n_members)  # shape: (N, members)

    # 2. Filter out NaNs manually (critical!)
    valid_mask = ~np.isnan(obs) & ~np.isnan(fct).any(axis=1)
    obs_clean = obs[valid_mask]
    fct_clean = fct[valid_mask]

    # 3. Calculate ranks: how many ensemble members are LESS than the observation?
    # (This is the core of a rank histogram)
    ranks = (fct_clean < obs_clean[:, np.newaxis]).sum(axis=1)

    # 4. Create the histogram bins (0 to n_members)
    counts_np, _ = np.histogram(ranks, bins=np.arange(n_members + 2))

    # 5. Convert back to xarray just for your plotting code to work
    counts = xr.DataArray(counts_np, coords={"rank": np.arange(n_members + 1)}, dims="rank")
    # 4. Normalize to Relative Frequency
    # This makes the y-axis a probability (0.0 to 1.0) rather than a raw count (e.g., 500,000)
    freq = counts / counts.sum()

    # Calculate Reliability Score (Root Mean Square Error from Ideal)
    # A perfectly flat histogram will have a score of 0.
    rel_score = np.sqrt(((freq - ideal_freq) ** 2).mean())
    dynamic_lw = 0.2 if n_members > 30 else 0.7
    # Use ['rank'] to get the coordinate, and .values to pass a numpy array to matplotlib
    ax.bar(
        freq["rank"].values,
        freq.values,
        color="skyblue",
        edgecolor="black",
        linewidth=dynamic_lw,
        width=1.0,
        align="center",
    )
    # Add the dynamic "Perfect" line
    ax.axhline(y=ideal_freq, color="red", linestyle="--", label="Ideal")

    # Aesthetics
    ax.set_title(f"{title} (N={n_members})")
    # Add Reliability Index text box
    ax.text(
        0.5,
        0.8,
        f"Reliability Index\n(lower better): {rel_score.values:.3f}",
        transform=ax.transAxes,
        fontsize="small",
        va="top",
        ha="center",
    )
    ax.set_xlabel("Rank")
    ax.set_ylabel("Relative Frequency")

    # Important: Center the ticks if you have fewer members (like N=10)
    ax.set_xticks(range(n_members + 1))
    # Dynamic Y-Limit to focus on the deviations
    # We set the top to 2.5x the ideal frequency so U-shapes are visible but consistent
    ax.set_ylim(0, ideal_freq * 2.5)

    # Only show every 5th or 10th tick on x-axis if there are too many members (e.g. 64)
    if n_members > 30:
        ax.set_xticks(range(0, n_members + 2, 10))

    ax.legend(loc="upper center")

# Remove empty subplot
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
ofile = PLOTDIR / f"rank_histogram_{isobaricInhPa}{shortName}_{years}.png"
plt.savefig(ofile, dpi=200)
print(ofile)

In [ ]:
for ds_model, title in zip(
    [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext],
    ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"],
):
    print(title, ds_model.nbytes / 1024 / 1024 / 1024)

In [ ]:
if False:
    # calculate deviation from the ensemble mean (anomaly)
    ds_fengwu_anom = ds_fengwu - ds_fengwu.isel(number=slice(0, member_cutoff)).mean(dim="number")
    # inflate the spread about the mean
    ds_fengwu_inflated_spread = (
        ds_fengwu.isel(number=slice(0, member_cutoff)).mean(dim="number") + 3 * ds_fengwu_anom
    )

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(14, 8), sharex=True, sharey=True)

for ax, ds_model, title in zip(
    axes.flat,
    [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext],
    ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"],
):
    print(title)
    # Select only the first 5 ensemble members for evaluation
    ds_model = ds_model[shortName].isel(number=slice(0, member_cutoff))

    # Assign 'day' coordinate for plotting
    ds_model = ds_model.assign_coords(day=ds_model.forecast_hour / 24)

    # Align the truth dataset to the forecast's valid_time for accurate comparison
    # This ensures that for each forecast point, we have a corresponding observation.
    aligned_truth = truth[shortName].sel(valid_time=ds_model["valid_time"])

    # --- RMSE and Spread Calculation (Original) ---
    # Calculate ensemble spread: std dev across members, then averaged
    ensemble_spread = ds_model.std(dim="number", ddof=1).mean(
        dim=["initialization_time", "latitude", "longitude"]
    )

    # Calculate RMSE: error of ensemble mean vs. truth
    error = ds_model.mean(dim="number") - aligned_truth
    se = error**2
    mse = se.mean(dim=["latitude", "longitude"])
    rmse = np.sqrt(mse).mean(dim="initialization_time")

    crps = xr.apply_ufunc(
        ps.crps_ensemble,  # The function to apply
        aligned_truth,  # Argument 1 (observations)
        ds_model,  # Argument 2 (forecasts)
        input_core_dims=[[], ["number"]],  # 'number' is the ensemble dim in forecast
        join="inner",  # handle coord mismatches
        dask="parallelized",  # Allows Dask to handle it in chunks
        output_dtypes=[float],
        dask_gufunc_kwargs={"allow_rechunk": True},
    )

    crps_mean = crps.mean(dim=["initialization_time", "latitude", "longitude"]).compute()

    # --- Plotting ---
    # Plot the original and new metrics
    rmse.plot.line(ax=ax, x="day", marker="o", label="RMSE")
    ensemble_spread.plot.line(ax=ax, x="day", marker="o", label="Spread")
    crps_mean.plot.line(ax=ax, x="day", marker="o", label="CRPS", linestyle="--")

    # --- Formatting ---
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.set_title(
        f"{title} {member_cutoff}-mem cutoff\n{isobaricInhPa}-hPa {shortName}",
        fontsize=12,
    )
    ax.legend()
    ax.set_ylabel(f"Score [{units}]")
    ax.set_xlabel("Forecast Lead Time (days)")

# Adjust layout
plt.tight_layout()

ofile = (
    PLOTDIR
    / f"rmse_spread_crps_{member_cutoff}-member_cutoff_{isobaricInhPa}{shortName}_{years}.png"
)
plt.savefig(ofile, dpi=200)
print(ofile)

In [ ]:
# Define the models and their titles for the legend
datasets = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

# --- Create Subplots: One for each metric ---
fig, axes = plt.subplots(ncols=4, figsize=(16, 5))
ax_rmse, ax_spread, ax_ratio, ax_crps = axes

# Variable to store the color of the 'Fengwu' plot
fengwu_color = None

z = SurfaceGeoPotential(cache=False)([0])
sfc_hgt_cutoff = 2000

# --- Loop through models, calculate metrics, and plot on the correct panel ---
for ds_model, title in zip(datasets, titles):
    print(title)
    # Select only the first 5 ensemble members for evaluation
    ds_model = ds_model.isel(number=slice(0, member_cutoff))

    # Assign 'day' coordinate for plotting
    ds_model = ds_model.assign_coords(day=ds_model.forecast_hour / 24)
    # Mask surface >= 2000m
    ds_model = ds_model.where(
        z.sel(lat=ds_model.latitude, lon=ds_model.longitude).squeeze() < 2000 * g.m
    )

    # Align the truth dataset to the forecast's valid_time for accurate comparison
    aligned_truth = truth[shortName].sel(valid_time=ds_model["valid_time"])
    # --- RMSE and Spread Calculation ---
    ensemble_spread = (
        ds_model[shortName]
        .std(dim="number", ddof=1)
        .mean(dim=["initialization_time", "latitude", "longitude"])
    )
    error = ds_model.mean(dim="number") - aligned_truth
    se = error[shortName] ** 2
    mse = se.mean(dim=["latitude", "longitude"])
    rmse = np.sqrt(mse).mean(dim="initialization_time")
    ratio = ensemble_spread/rmse

    crps = xr.apply_ufunc(
        ps.crps_ensemble,  # The function to apply
        aligned_truth,  # Argument 1 (observations)
        ds_model[shortName],  # Argument 2 (forecasts)
        input_core_dims=[[], ["number"]],  # 'number' is the ensemble dim in forecast
        join="inner",  # handle coord mismatches
        dask="parallelized",  # Allows Dask to handle it in chunks
        output_dtypes=[float],
        dask_gufunc_kwargs={"allow_rechunk": True},
    )

    crps_mean = crps.mean(dim=["initialization_time", "latitude", "longitude"]).compute()

    # --- Plotting Logic ---
    plot_kwargs = {"marker": "o", "label": title, "alpha": 0.7}

    # Check if the current model is the one to be dashed
    if title == "Fengwu spreadx3":
        plot_kwargs["linestyle"] = "--"
        # Use the stored color from the 'Fengwu' plot
        if fengwu_color:
            plot_kwargs["color"] = fengwu_color

    # Plot each metric and store the color if it's the 'Fengwu' model
    line_rmse = rmse.plot.line(ax=ax_rmse, x="day", **plot_kwargs)
    line_spread = ensemble_spread.plot.line(ax=ax_spread, x="day", **plot_kwargs)
    line_ratio = ratio.plot.line(ax=ax_ratio, x="day", **{**plot_kwargs, "marker": "s"})
    line_crps = crps_mean.plot.line(ax=ax_crps, x="day", **plot_kwargs)

    # If we just plotted 'Fengwu', get its color for the next iteration
    if title == "Fengwu":
        # The plot function returns a list of lines; get the color from the first one
        fengwu_color = line_rmse[0].get_color()



# --- Formatting for all subplots ---
for ax in axes:
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.legend(title=f"{member_cutoff}-mem cutoff", fontsize="small")
    ax.set_xlabel("Forecast Lead Time (days)")

# Set titles and y-axis label for each panel
ax_rmse.set_title("RMSE")
ax_spread.set_title("Spread")
ax_ratio.set_title("Spread/Error Ratio")
# Reference line for perfect spread
ax_ratio.axhline(1, color="red", linestyle="-", alpha=0.3)
ax_ratio.set_ylim(0.1, 2)  # Ratios usually hover around 1.0
ax_crps.set_title("CRPS")

ax_rmse.set_ylabel(f"Score [{units}]")

plt.suptitle(f"{isobaricInhPa}-hPa {shortName}")
plt.tight_layout()

plt.savefig(
    PLOTDIR
    / f"rmse_spread_crps_all_models_on_same_panel_lt{sfc_hgt_cutoff}m_{member_cutoff}-mem_cutoff_{isobaricInhPa}{shortName}_{years}.png",
    dpi=200,
)

In [ ]:
truth

In [ ]:
# Initialize climatology source - defaults to 1990-2020 mean
climo_src = WB2Climatology(verbose=False)

# 2. Calculate Truth Anomaly
# fetch() returns data aligned to the provided times
wb2_var = f"{shortName}{isobaricInhPa}"
ds_climo = await climo_src.fetch(pd.to_datetime(truth.valid_time), wb2_var)
ds_climo = ds_climo.squeeze("variable")
ds_climo = ds_climo.rename({"time": "valid_time", "lat": "latitude", "lon": "longitude"})
if shortName == "z":
    # it's geopotential; divide by gravity to get height
    ds_climo = ds_climo / g.m

In [ ]:
anom_truth = truth[shortName] - ds_climo

# Create a single figure and axes object
fig, ax = plt.subplots(figsize=(9, 6))

# Loop through each model to calculate and plot its ACC
for ds_model, title in zip(
    [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext],
    ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"],
):
    print(title)
    ds_model = ds_model[shortName].sel(number=slice(None, member_cutoff)).mean(dim="number")

    # Calculate the anomaly
    # Crucial to select valid_time from ds_climo, or 
    # initialization_time and valid_time will not match,
    # and automatic broadcasting across both will blow up memory.
    anom = ds_model - ds_climo.sel(valid_time=ds_model.valid_time)

    # Calculate the Anomaly Correlation Coefficient (ACC)
    acc = xr.corr(
        anom,
        anom_truth.sel(valid_time=anom.valid_time),
        dim=["latitude", "longitude", "initialization_time"],
    )
    # Convert forecast_hour to days for the x-axis
    acc = acc.assign_coords(day=acc.forecast_hour / 24)

    # Plot the ACC line on the single axes, using the model title as the label
    line_acc = acc.plot.line(ax=ax, x="day", marker="o", label=title, alpha=0.7)

ax.set_title(f"Forecast Comparison: {isobaricInhPa}-hPa {shortName}")

# Set labels for the axes
ax.set_ylabel("ACC")
ax.set_xlabel("Forecast Day")

ax.set_ylim(bottom=0.65, top=1.02)

# Add a grid and a legend
ax.grid(True, linestyle="--", alpha=0.6)
ax.legend(title=f"member cutoff: {member_cutoff}")

# Adjust layout to prevent labels from overlapping
plt.tight_layout()

ofile = PLOTDIR / f"acc.{shortName}{isobaricInhPa}.{member_cutoff}memcutoff.png"
fig.savefig(ofile)
print(ofile)

In [ ]:
# Difference between WeatherBench2 climo (1990-2019) and my climo (1959-1988)
my_climo = (
    xr.open_dataset("/glade/work/ahijevyc/share/ACC/hgt.4Xday.1959-1988.ltm.nc")["hgt"]
    .rename(lat="latitude", lon="longitude", time="valid_time")
    .sel(level=isobaricInhPa, **geographic_sel)
)


def add_day_hour(da):
    da["day_hour"] = da.valid_time.dt.dayofyear * 100 + da.valid_time.dt.hour
    return da


ds_climo = add_day_hour(ds_climo)
my_climo = add_day_hour(my_climo)
my_climo = my_climo.swap_dims(valid_time="day_hour").drop_vars("valid_time")

ds_climo = ds_climo.sel(latitude=my_climo.latitude, longitude=my_climo.longitude)
my_climo = my_climo.sel(day_hour=ds_climo.day_hour)

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax_args = {"projection": ccrs.PlateCarree()}

# Create the 3-panel layout (2 rows, 2 columns)
ax1 = plt.subplot(2, 2, 1, **ax_args)  # Top Left
ax2 = plt.subplot(2, 2, 2, **ax_args)  # Top Right
ax3 = plt.subplot(2, 2, (3, 4), **ax_args)  # Bottom (spanning both columns)

# 2. Define plotting parameters
plot_data = [
    (ds_climo.mean("valid_time"), ax1, "WeatherBench2 (1990-2020)", "viridis"),
    (my_climo.mean("valid_time"), ax2, "My Climo (1980-2010)", "viridis"),
    (diff.mean("valid_time"), ax3, "Difference (WB2 - My)", "RdBu_r"),
]

# 3. Iterate and plot
for data, ax, title, cmap in plot_data:
    p = data.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, add_colorbar=True)
    ax.coastlines()
    ax.set_title(title, fontweight="bold")

plt.tight_layout()
plt.show()

In [ ]:
# --- Main Logic ---
datasets = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

# --- 1. Pre-calculate all RMSE arrays to find a global color scale ---
all_rmse_maps = []
for ds_model, title in zip(datasets, titles):
    # Align the truth dataset to the forecast's valid_time
    aligned_truth = truth.sel(valid_time=ds_model["valid_time"], method="nearest")

    # Calculate error and RMSE, but keep spatial dimensions
    error = ds_model.mean(dim="number") - aligned_truth
    se = error[shortName] ** 2
    # Average over initialization time to get a stable RMSE map for each forecast hour
    mse = se.mean(dim=["initialization_time"])
    rmse = np.sqrt(mse)
    all_rmse_maps.append(rmse)

# Select the forecast hour to plot (e.g., the last one)
fhr_to_plot = ds_mpas.forecast_hour.values[-1]
rmse_maps_at_fhr = [rmse.sel(forecast_hour=fhr_to_plot) for rmse in all_rmse_maps]

# Determine a robust color range using percentiles to avoid outliers skewing the scale
# First, combine all data into a single array
all_rmse_values = np.concatenate([da.values.ravel() for da in rmse_maps_at_fhr])
all_rmse_values = all_rmse_values[~np.isnan(all_rmse_values)]
# Then calculate the 2nd and 98th percentiles
vmin = np.percentile(all_rmse_values, 1)
vmax = np.percentile(all_rmse_values, 99.5)

# --- 2. Create the plots ---
fig, axes = plt.subplots(
    ncols=3,
    nrows=3,
    figsize=(15, 8),
    subplot_kw={
        "projection": ccrs.LambertConformal(central_longitude=-98.0, central_latitude=39.5)
    },
)

for i, (ax, ds_model, title) in enumerate(zip(axes.flat, datasets, titles)):
    rmse_map = rmse_maps_at_fhr[i]

    # Plot the 2D RMSE map
    im = rmse_map.plot.pcolormesh(
        ax=ax,
        transform=ccrs.PlateCarree(),
        vmin=vmin,
        vmax=vmax,
        cmap="viridis",
        add_colorbar=False,  # We will add a single colorbar later
    )

    # Add map features
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.STATES, edgecolor="gray", lw=0.5)

    # Set title for each subplot
    ax.set_title(title)
    ax.set_xlabel("")
    ax.set_ylabel("")

    extent = [-124, -72, 25, 50]
    ax.set_extent(extent, crs=ccrs.PlateCarree())

# Hide any unused subplots
for i in range(len(datasets), len(axes.flat)):
    axes.flat[i].axis("off")

# Add a single, shared colorbar for the entire figure
fig.colorbar(
    im,
    ax=axes,
    orientation="vertical",
    label=f"RMSE of {isobaricInhPa}{shortName} [{units}]",
    shrink=0.8,
    pad=0.02,
)

# Adjust layout
plt.suptitle(f"RMSE of {isobaricInhPa}{shortName} at Forecast Hour {fhr_to_plot}", fontsize=16)
plt.savefig(PLOTDIR / f"spatial_rmse_{isobaricInhPa}{shortName}_{years}.png", dpi=200)

In [ ]:
# --- 1. Pre-calculate all Spread arrays to find a global color scale ---
all_spread_maps = []
for ds_model, title in zip(datasets, titles):
    # Calculate ensemble spread, keeping spatial dimensions
    # Average over initialization_time to get a stable spread map for each forecast hour
    ensemble_spread = (
        ds_model[shortName].std(dim="number", ddof=1).mean(dim=["initialization_time"])
    )
    all_spread_maps.append(ensemble_spread)

# Select the forecast hour to plot (e.g., the last one)
fhr_to_plot = ds_mpas.forecast_hour.values[-1]
spread_maps_at_fhr = [spread.sel(forecast_hour=fhr_to_plot) for spread in all_spread_maps]

# Determine a robust color range using percentiles to avoid outliers skewing the scale
# First, combine all data into a single array
all_spread_values = np.concatenate([da.values.ravel() for da in spread_maps_at_fhr])
all_spread_values = all_spread_values[~np.isnan(all_spread_values)]
# Then calculate the 1st and 99.5th percentiles
vmin = np.percentile(all_spread_values, 1)
vmax = np.percentile(all_spread_values, 99.5)


# --- 2. Create the plots ---
fig, axes = plt.subplots(
    ncols=3,
    nrows=3,
    figsize=(15, 8),
    subplot_kw={
        "projection": ccrs.LambertConformal(central_longitude=-98.0, central_latitude=39.5)
    },
)

for i, (ax, ds_model, title) in enumerate(zip(axes.flat, datasets, titles)):
    spread_map = spread_maps_at_fhr[i]

    # Plot the 2D Spread map
    im = spread_map.plot.pcolormesh(
        ax=ax,
        transform=ccrs.PlateCarree(),
        vmin=vmin,
        vmax=vmax,
        cmap="viridis",
        add_colorbar=False,  # We will add a single colorbar later
    )

    # Add map features
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.STATES, edgecolor="gray", lw=0.5)

    # Set title for each subplot
    ax.set_title(title)
    ax.set_xlabel("")
    ax.set_ylabel("")

    extent = [-124, -72, 25, 50]
    ax.set_extent(extent, crs=ccrs.PlateCarree())

# Hide any unused subplots
for i in range(len(datasets), len(axes.flat)):
    axes.flat[i].axis("off")

# Add a single, shared colorbar for the entire figure
# Note: colorbar is attached to the whole figure, not just the populated axes
fig.colorbar(
    im,
    ax=axes.ravel().tolist(),
    orientation="vertical",
    label=f"Spread of {isobaricInhPa}{shortName} [{units}]",
    shrink=0.8,
    pad=0.02,
)

# Adjust layout
plt.suptitle(f"Spread of {isobaricInhPa}{shortName} at Forecast Hour {fhr_to_plot}", fontsize=16)
plt.savefig(PLOTDIR / f"spatial_spread_{isobaricInhPa}{shortName}_{years}.png", dpi=200)

In [ ]:
# --- Configuration ---
target_init_time = ds_mpas.initialization_time.values[0] 
fhr_to_plot = 120 
levels = [5400, 5640, 5880] 

datasets = [ds_mpas, ds_fv3, ds_gefs, ds_pangu, ds_fengwu, ds_graphcast, ds_wxnext]
titles = ["MPAS", "FV3", "GEFS", "Pangu", "Fengwu", "GraphCast", "WxNext"]

# --- Create the plots ---
fig, axes = plt.subplots(
    ncols=3,
    nrows=3,
    figsize=(16, 10), # Adjusted height for tighter look
    subplot_kw={
        "projection": ccrs.LambertConformal(central_longitude=-98.0, central_latitude=39.5)
    },
)

for i, (ax, ds_model, title) in enumerate(zip(axes.flat, datasets, titles)):
    # 1. Select Slices
    ds_slice = ds_model.sel(initialization_time=target_init_time, forecast_hour=fhr_to_plot, method="nearest")
    ens_mean = ds_slice[shortName].mean(dim="number")
    target_valid_time = ds_slice.valid_time.values
    truth_slice = truth.sel(valid_time=target_valid_time, method="nearest")[shortName]

    # 2. Calculate RMSE
    error_sq = (ens_mean - truth_slice)**2
    spatial_rmse = np.sqrt(error_sq.mean().values)

    # 3. Plot Ensemble Members (Gray)
    for mem in ds_slice.number.values:
        ax.contour(
            ds_slice.longitude, ds_slice.latitude, ds_slice.sel(number=mem)[shortName],
            levels=levels, colors='gray', linewidths=0.3, alpha=0.8, transform=ccrs.PlateCarree()
        )

    # 4. Plot Ensemble Mean (Black) & Add Labels
    mean_contour = ax.contour(
        ds_slice.longitude, ds_slice.latitude, ens_mean,
        levels=levels, colors='black', linewidths=2.0, transform=ccrs.PlateCarree()
    )
    # Inline labels for the mean only
    ax.clabel(mean_contour, inline=True, fontsize=9, fmt='%d', colors='black')
    
    # 5. Plot Truth (Red)
    ax.contour(
        truth_slice.longitude, truth_slice.latitude, truth_slice,
        levels=levels, colors='red', linestyles='--', linewidths=1.5, transform=ccrs.PlateCarree()
    )

    # --- Map Aesthetics ---
    ax.coastlines(resolution='110m', linewidth=0.8)
    ax.add_feature(cfeature.STATES, edgecolor="gray", lw=0.3)
    ax.set_extent([-122, -74, 25, 48], crs=ccrs.PlateCarree()) # Slightly tighter extent
    
    ax.set_title(f"{title} (RMSE: {spatial_rmse:.1f})", fontsize=11, pad=5)

# Cleanup
for i in range(len(datasets), len(axes.flat)):
    axes.flat[i].axis("off")

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='gray', lw=1, label='Members'),
    Line2D([0], [0], color='black', lw=2, label='Ens Mean'),
    Line2D([0], [0], color='red', lw=1.5, ls='--', label='Truth')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, frameon=False, bbox_to_anchor=(0.5, 0.07))

# --- Tighten Layout ---
# wspace: width between subplots, hspace: height between subplots
plt.subplots_adjust(left=0.05, right=0.95, bottom=0.12, top=0.90, wspace=0.02, hspace=0.15)

plt.suptitle(f"{isobaricInhPa} {shortName} Spaghetti Plot | Init: {target_init_time} | Fhr {fhr_to_plot}", fontsize=14, y=0.96)
plt.show()

In [ ]:
import zarr

o = zarr.open("gefs.850t.[2023, 2024].240h.zarr")
o["t"].info

In [ ]:
z = zarr.open("gefs.500z.[2023, 2024].240h.zarr")
z["z"].info